# 14-9. 프로젝트 - Movielens 영화 추천 실습

1. 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
2. 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
3. 별점을 시청횟수로 해석해서 생각하겠습니다.
4. 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

## 1-1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, 
                      engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()


,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

print("뿡=3")

뿡=3


In [4]:
# ratings['user_id', 'movie_id', 'counts', 'timestamp']
ratings["counts"]

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
ratings.head()

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [11]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [ ]:
# ratings : user_id, movie_id, ratings, timestamp
# movies  : movie_id, title, genre
    
    
    
    
    
    

### 데이터 둘러보기

In [7]:
# 유저 수
ratings['user_id'].nunique()

6039

In [8]:
# 영화 수 in ratings
ratings['movie_id'].nunique()

3628

In [9]:
# 영화 수 in movies
movies['movie_id'].nunique()

3883

In [10]:
# 인기 많은 영화
ratings_count = ratings.groupby('movie_id')['user_id'].count()
ratings_count.sort_values(ascending=False).head(10)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
Name: user_id, dtype: int64

In [5]:
# pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.

# 유저 수
data['user_id'].nunique()

358868

In [6]:
# 아티스트 수
data['artist'].nunique()

291346

In [7]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [8]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [9]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

### 모델 검증을 위한 사용자 초기 정보 세팅

In [10]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zimin']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                        # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


### 모델에 활용하기 위한 전처리 (실습)

In [11]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

print("뿡=3")

In [12]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zimin'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['black eyed peas'])

358868
376


In [13]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()

if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


### 데이터 전처리 완료

In [14]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


## 14-5. Matrix Factorization(MF)
기본 아이디어는 아래 그림과 같습니다. (m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어 입니다.<br>
 대체로 k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리해집니다. 하지만 아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델입니다.<br><br>
큰 평가 행렬 R을 두 개의 Feature Matrix P와 Q로 분해.

## 14-6. CSR(Compressed Sparse Row) Matrix

In [15]:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
# where data, row_ind and col_ind satisfy 
# the relationship a[row_ind[k], col_ind[k]] = data[k].

# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))

csr_data # csr_data.toarray() 하면 데이터의 양이 많아서 kernel이 down 된다. 


<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

## 14-7. MF 모델 학습하기
이전 스텝에서 설명한 Matrix Factorization 모델을 **implicit 패키지**(https://github.com/benfred/implicit)를 사용하여 학습해 봅시다.<br><br>
implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.<br><br>
이 패키지에 구현된 **als(AlternatingLeastSquares) 모델**을 사용하겠습니다. Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, <br>
한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 **AlternatingLeastSquares 방식**이 효과적인 것으로 알려져 있습니다.

In [22]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# **********************************************************************
# #implicit의 Basic Usage
# import implicit
# # initialize a model
# model = implicit.als.AlternatingLeastSquares(factors=50)
# # train the model on a sparse matrix of user/item/confidence weights
# model.fit(user_item_data)
# # recommend items for a user
# recommendations = model.recommend(userid, user_item_data[userid])
# # find related items
# related = model.similar_items(itemid)
# **********************************************************************

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'


# AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다.

# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
# 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
# 3. use_gpu : GPU를 사용할 것인지
# 4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

# 1과 4를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.

# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.01, 
                                    use_gpu=False, 
                                    iterations=15, 
                                    dtype=np.float32)
print("뿡=3")

=3


In [23]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [27]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print("뿡=3")

뿡=3


In [28]:
# zimin vecor의 내용
zimin_vector

array([ 0.01169535,  0.15312137, -0.21560195, -1.7101544 ,  0.36993378,
        0.81515414,  0.51849556,  1.6747963 ,  1.2985485 ,  0.10534978,
        0.4456235 , -0.82636184, -1.8274918 , -0.8060825 , -0.26139963,
        1.0623056 ,  0.52284694, -0.4203953 , -0.23516458,  0.68490195,
        0.04867486,  0.49998626,  0.03806423,  0.1503468 , -0.13769679,
        0.18871231, -0.39560026,  0.5922866 ,  1.0006877 , -1.56305   ,
        0.7176193 ,  0.16783175,  0.00312326,  1.2713927 , -0.2816705 ,
        0.11726557,  0.6553673 , -0.04325461,  0.25515795, -0.65945935,
        0.21088687, -0.7917076 , -0.16676019, -0.95500714,  0.2913208 ,
        0.00572771, -1.6197425 , -0.4654501 ,  1.1927453 , -0.5944834 ,
       -0.09736005, -1.0136906 ,  0.3141982 ,  0.02812445,  0.40308833,
        1.1441479 ,  1.2287925 ,  0.13958947,  0.02099192, -1.1159548 ,
       -0.11113559,  0.13077542, -0.04258401,  1.4145259 , -0.7306547 ,
       -0.24515754,  0.20134956, -0.5049722 , -0.23581071,  0.38

In [29]:
# black_eyed_peas vector의 내용
black_eyed_peas_vector

array([ 0.02152998,  0.01312743, -0.00221638, -0.01315212,  0.01683473,
        0.02317275,  0.00672522,  0.02285541,  0.00425628,  0.00841868,
        0.01276143,  0.00511517, -0.02066208, -0.00454535,  0.00622293,
        0.00310078,  0.01848973,  0.00152681, -0.00232819,  0.01924854,
        0.00469799,  0.01620649,  0.01450966,  0.00393891,  0.0229918 ,
        0.00258313, -0.01704722,  0.02376237,  0.01702951, -0.02020077,
        0.01993771,  0.02288953,  0.00461423,  0.01133724,  0.00034839,
       -0.00259494,  0.00877318,  0.00422962,  0.01678957,  0.00762755,
        0.00969152,  0.00247916,  0.02804877,  0.0047988 ,  0.01257185,
        0.00252715, -0.01477236, -0.00035118,  0.02907138,  0.00640605,
        0.00644223, -0.00133874,  0.00502729, -0.00010455,  0.00046094,
        0.01282149,  0.02756918,  0.01546468,  0.01864011,  0.00479144,
        0.00566057, -0.00390746, -0.00305116,  0.01567552, -0.01198488,
       -0.00100792,  0.00230715, -0.00346667,  0.00765505,  0.00

In [30]:
# zimin과 black_eyed_peas를 내적하는 코드 (두 벡터를 곱하면 어떤 값이 나오나???)
np.dot(zimin_vector, black_eyed_peas_vector)

0.5194117

In [31]:
# 훈련된 모델이 저자의 queen에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다.
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.32845467

## 14-8. 비슷한 아티스트 찾기 + 유저에게 추천하기
**AlternatingLeastSquares** 클래스에 구현되어 있는 **similar_items** 메서드를 통하여 비슷한 아티스트를 찾습니다. 

In [32]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15) # "N=15"는 뭐지???
similar_artist

# (아티스트의 id, 유사도) Tuple 로 반환하고 있음

[(62, 0.9999999),
 (277, 0.98796785),
 (5, 0.97800094),
 (217, 0.9757673),
 (28, 0.97421676),
 (473, 0.9689929),
 (247, 0.9680395),
 (490, 0.9573791),
 (910, 0.9535752),
 (694, 0.9527422),
 (418, 0.952665),
 (1018, 0.95125544),
 (782, 0.9484199),
 (268, 0.9445488),
 (531, 0.94326836)]

In [33]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'red hot chili peppers',
 'radiohead',
 'the killers',
 'placebo',
 'the beatles',
 'oasis',
 'nirvana',
 'foo fighters',
 'u2',
 'the smashing pumpkins',
 'the white stripes',
 'pink floyd',
 'depeche mode']

In [34]:
# 마니아들의 존재로 인해 같은 장르의 아티스트들의 벡터들도 더 가까워져서 
# get_similar_artist시 장르별 특성이 두드러지게 될 것입니다.

def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [35]:
get_similar_artist('2pac') # 힙합족 장르가 조회 된다.

['2pac',
 'snoop dogg',
 'notorious b.i.g.',
 'nas',
 'jay-z',
 'dr. dre',
 'the game',
 '50 cent',
 'common',
 'ludacris']

In [36]:
get_similar_artist('lady gaga') # 여자가수가 주로 나온다.

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'the pussycat dolls',
 'beyoncé',
 'christina aguilera',
 'kelly clarkson',
 'leona lewis',
 'justin timberlake']

### 유저에게 아티스트 추천하기

In [37]:
# recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받습니다. 
# filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument

user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, 
                                         csr_data, 
                                         N=20, 
                                         filter_already_liked_items=True)
artist_recommended

[(350, 0.44990414),
 (409, 0.43966192),
 (550, 0.4312501),
 (1800, 0.42714602),
 (627, 0.42415386),
 (274, 0.42213315),
 (369, 0.42014503),
 (2249, 0.4116193),
 (564, 0.39748943),
 (354, 0.3866514),
 (391, 0.37805754),
 (24, 0.3770157),
 (724, 0.37498093),
 (621, 0.37002248),
 (5555, 0.36997238),
 (247, 0.3636998),
 (355, 0.36044082),
 (901, 0.35896534),
 (944, 0.3563815),
 (5, 0.35432753)]

In [38]:
[idx_to_artist[i[0]] for i in artist_recommended]

['rihanna',
 'amy winehouse',
 'britney spears',
 'lady gaga',
 'maroon 5',
 'michael jackson',
 'justin timberlake',
 'katy perry',
 'kanye west',
 'nelly furtado',
 'christina aguilera',
 'jack johnson',
 'lily allen',
 'alicia keys',
 'timbaland',
 'the beatles',
 'madonna',
 'pink',
 'avril lavigne',
 'red hot chili peppers']

In [40]:
# 모델은 왜 rihanna를 추천해 줬을까요? 
# AlternatingLeastSquares 클래스에 구현된 explain 메서드를 사용하면 제가 기록을 남긴 데이터 중 
# 이 추천에 기여한 정도를 확인할 수 있습니다.

rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.21749321327321441),
 ('black eyed peas', 0.1377909029131906),
 ('jason mraz', 0.05328783360280735),
 ('coldplay', 0.043801738445244764),
 ('maroon5', 5.5783867343096085e-05)]